# Pacemaker

*  Pacemakerとは
    * HA(高可用性)クラスタソフト
      * 故障を検知した場合，待機系へサービスを引き継がせること（フェイルオーバ）によってサービスのダウンタイムを最小限にする事が可能
      * Heartbeatの後継ソフトウェア
      * リソース制御機能のみ。クラスタ制御機能を持つソフトウェアと組み合わせる必要がある(Heartbeat,Corosyncなど)

* 歴史
      * 2003 Heartbeat ver1 : 相手ノードの監視など、クラスタ制御機能のみ
      * 2005 Heartbeat ver2 : リソース制御機能（CRM：Cluster Resource Manager)追加
      * 2008 Pacemaker ver1 : Heartbeatからリソース制御機能を独立させる
      * ???? Heartbert ver3 : クラスタ制御機能のみメンテナンスされている

## 基本機能

* クラスタ制御機能 (Heartbeat担当)
   * 機能例
     * クラスタ状態を伝播する通信
     * クラスタへのノード参加判断
     * クラスタノード間における情報の同期
     * 一定間隔で相手ノードと通信し生死を確認
   * ハートビート通信
     * 生死確認の通信はインターコネクト通信のこと
       * これが途絶えたときは相手が故障したと判断し，フェイルオーバ処理を実施

* リソース制御機能 (Pacemaker担当)
 * 一定間隔でリソースを監視し，正常動作していないと判断した場合にリソースのフェイルオーバ処理を行う
   * リソース
     * Pacemakerが制御対象とするアプリケーション(Apache,MySQLなど)，ネットワーク，ディスク等のこと
     * リソースをリソースエージェントを介して起動（start），停止（stop），監視（monitor）を実行しリソース制御を行う
   * リソースエージェント
     * リソースとPacemakerを仲介するプログラムで，主にシェルスクリプトで作成されている。自作もできる。

## リソースの種類

* Primitive
    * 通常のAct-Standby構成で用いるリソース
    * どこか一カ所のノードで動くことができる
        * クラスタ全体のある1ノードだけで動いていればよいリソースに使用
        * リソースが故障すれば、他のノードにフェールオーバーさせることができる
    * データベースやメールサーバのようなものをHAクラスタリングする場合は通常このリソースを定義
* Clone
    * Primitive リソースを複数のノードで動作させたい場合に使用
    * 定義方法
        * まずPrimitive を定義し、それをClone化する
* Master/Slave
    * Clone リソースをさらに発展させたもの
        * Cloneリソースで親子の関係があるリソースに使用
    * 定義方法
        * まずPrimitive を定義
        * Master/Slave化する
    * リソースの起動・停止操作
        * start, stop 
            * Primitive, Clone 用のRAに実装されている
        * promote(昇格), demote(降格)
            * <font color="red">親に昇格させたり子供に降格させる</font> Master/Slave専用の操作
* Group
    * 複数のPrimitiveリソースを１つのグループとしてまとめたリソース
    * 同じグループとなったPrimitiveリソースは<font color="red">必ず同じノードで、定義された順番に起動／停止</font>
        * Pacemaker内部の動作としては「colocation(同居)制約」と「order(順序)制約」の両制約をかけたのと同等
        

## 参考

* [第1回　Pacemakerの歴史を見てみよう！](http://gihyo.jp/admin/serial/01/pacemaker/0001?page=2)


## crmのshowコマンド解説

```
crm(live)configure# show
================================================
node $id="10c76ad1-e4c4-441d-a62e-bec05e25f882" drbd01
node $id="ea4ec1c7-bdd3-4bf7-9177-c8d2a1569389" drbd02


### primitiveコマンド書式
primitive リソースID RA(リソースエージェント)名  [params RAに渡すパラメータ...] [op start|stop|monitor オペレーション時の設定...]

---------
primitive res_drbd0 ocf:linbit:drbd \
        params drbd_resource="r0" drbdconf="/etc/drbd.conf" \
        op monitor interval="20s"

※ pacemakerでdrbdの起動や停止を制御する。chkconfig drbd offにしておくこと。
https://www.drbd.jp/users-guide/s-pacemaker-crm-drbd-backed-service.html


リソースID : res_drbd0
RA名 : ocf:linbit:drbd (/usr/lib/ocf/resource.d/linbit/drbdに実体のシェルスクリプトがある)
params
  drbd_resource="r0" : drbdのリソースを指定
  drbdconf="/etc/drbd.conf" :  drbdのconfファイルを指定

シェルスクリプト中でこれらのパラメータが利用される
### /usr/lib/ocf/resource.d/linbit/drbd
...

<parameters>
...
<parameter name="drbd_resource" unique="1" required="1">
...
</parameters>

...

<parameters>
...
<parameter name="drbdconf">
...
</parameters>
...

op monitor interval="20s" : オペレーションの繰り返し間隔


---------
primitive res_filesystem ocf:heartbeat:Filesystem \
        params device="/dev/drbd0" fstype="ext4" directory="/var/share" \
        op monitor interval="20s"

※ pacemakerでマウントを制御する。ここで制御するファイルシステムは/etc/fstabに書いてはいけない

リソースID : res_filesystem
RA名 : ocf:heartbeat:Filesystem (/usr/lib/ocf/resource.d/heartbeat/Filesystemに実体のシェルスクリプトがある)
params
  device="/dev/drbd0" : Filesystemを指定
  fstype="ext4" : fstypeを指定
  directory="/var/share" : マウントポイントを指定

---------
primitive res_ip ocf:heartbeat:IPaddr \
        params ip="10.136.3.167" cidr_netmask="17" nic="eth0" \
        op monitor interval="10s"

※ VIPの付け替えを制御する。ここで制御するアドレスは/etc/sysconfig/network-scripts/ifcfg-eth0:1などのファイルを作ってはいけない。

リソースID : res_ip
RA名 : ocf:heartbeat:IPaddr (/usr/lib/ocf/resource.d/heartbeat/IPaddrに実体のシェルスクリプトがある)
params
  ip="10.136.3.167" : IPアドレスを指定
  cidr_netmask="17" : netmaskを指定
  nic="eth0" : nicを指定

---------
group rg_drbd res_ip res_filesystem

※ primitiveで定義した個々のリソースをグループ化する。グループ化すると以下の制御が行われる
   * 常に同一サーバで起動する。
   * groupコマンドに指定した順にリソースを起動する。またその逆順に停止する。

### groupコマンド書式
group グループID リソースID

グループID : rg_drbd
リソースID : res_ip, res_filesystem

つまり、VIPの付与とファイルシステムのマウントは同一のサーバ上で行われる

---------
ms ms_drbd0 res_drbd0 \
        meta master-max="1" master-node-max="1" clone-max="2" clone-node-max="1" notify="true"

※ msコマンドでmaster/slaveリソースを定義

### msコマンド書式
ms マスターID リソースID

マスターID : ms_drbd0
リソースID : res_drbd0
meta (msリソースの動作設定)
  master-max="1" : masterになれるリソース数
  master-node-max="1" : 1つのノードでmasterになれるリソース数
  clone-max="2" : 生成するインスタンス数 (2ノード構成であれば2を指定)
  clone-node-max="1" : 一つのノードで同時に起動するインスタンス数
  notify="true" : DRBDはnotifyアクションを使うのでtrueにする

http://www.slideshare.net/tatsuyaw/pacemaker-osc2014tokyo-31882542

---------
colocation c_rg_drbd_on_drbd0 inf: rg_drbd ms_drbd0:Master

リソースの同居制約を定義する

### colocationコマンド書式
colocation コロケーションID <score>: リソースA:役割 リソースB:役割 ...

コロケーションID : c_rg_drbd_on_drbd0
スコア : inf　リソースAが故障時にリソースBも影響を受ける  (スコアが0の場合は受けない)

rg_drbd ms_drbd0:Master

つまり、DRBDがMaster状態のサーバでrg_drbdが適用される(VIP付与、ファイルシステムマウント)
http://www.slideshare.net/tatsuyaw/pacemaker-osc2014tokyo-31882542

---------
order o_drbd_befor_rg_drbd inf: ms_drbd0:promote rg_drbd:start

### orderコマンド書式
order オーダーID <score>: リソースA:アクション リソースB:アクション ...

オーダーID : o_drbd_befor_rg_drbd
スコア : inf

リソースAが昇格(promote)してからリソースBを起動(start)(VIP付与、ファイルシステムマウント)
http://www.slideshare.net/tatsuyaw/pacemaker-osc2014tokyo-31882542


---------
property $id="cib-bootstrap-options" \
        dc-version="1.0.13-a83fae5" \
        cluster-infrastructure="Heartbeat" \
        stonith-enabled="false" \　 ### STONISH機能無効 http://gihyo.jp/admin/serial/01/pacemaker/0003
        no-quorum-policy="ignore" \ ### 2台構成のときはignoreでよい
        default-resource-stickiness="200" ###  通常は大きな値にして自動フェイルバックを無効にする http://linux-ha.osdn.jp/wp/archives/3786/3

http://gihyo.jp/admin/serial/01/pacemaker/0003

================================================
```

### 参考
* [動かして理解するPacemaker ～CRM設定編～ その２](http://linux-ha.osdn.jp/wp/archives/3855#primitive)
* [PacemakerのMaster/Slave構成の基本と事例紹介(DRBD、PostgreSQLレプリケーション) ](https://www.slideshare.net/tatsuyaw/pacemaker-osc2014tokyo-31882542)


## crmコマンド

* crm resource list
    * 全てのリソースのリストを取得する
    * crm resource statusと同じ

```
$ sudo crm resource list
================================================
 Resource Group: rg_drbd
     res_ip     (ocf::heartbeat:IPaddr):        Started
     res_filesystem     (ocf::heartbeat:Filesystem):    Started
 Master/Slave Set: ms_drbd0
     Masters: [ drbd01 ]
     Slaves: [ drbd02 ]
================================================
```

* crm resource demote (リソース)
    * リソースをSlaveに降格する

```
$ sudo crm resource demote ms_drbd0
$ sudo crm resource list
================================================
 Resource Group: rg_drbd
     res_ip     (ocf::heartbeat:IPaddr):        Stopped
     res_filesystem     (ocf::heartbeat:Filesystem):    Stopped
 Master/Slave Set: ms_drbd0
     Slaves: [ drbd01 drbd02 ]
================================================
```

* crm resource promote (リソース)
    * リソースをMasterに降格する

```
$ sudo crm resource promote ms_drbd0
$ sudo crm resource list
================================================
 Resource Group: rg_drbd
     res_ip     (ocf::heartbeat:IPaddr):        Started
     res_filesystem     (ocf::heartbeat:Filesystem):    Started
 Master/Slave Set: ms_drbd0
     Masters: [ drbd01 ]
     Slaves: [ drbd02 ]
================================================
```

### 参考
* [7.4 クラスタリソースの管理](https://www.suse.com/ja-jp/documentation/sle_ha/book_sleha/data/sec_ha_config_crm.html?view=print)

## crm_resourceコマンド

* crm_resource -r rg_drbd -M
    * MasterとSlaveを切り替える
    * マスターが現在の位置から別の位置に移動する
    * -rオプション
        * リソースを指定
* crm_resource -r rg_drbd -U
    * Masterに昇格できない制約を解除する
        * -MでMaster,Slaveを切り替えた場合group_masterが永遠に開始しなくなるため障害が発生しても自動で切り替わらない
        * -> -Uを実行して制約(location cli-standby-rg_drbd～)を解除する

## ステータス解読

```
============
Last updated: Wed Jul 19 13:13:58 2017
Stack: Heartbeat
Current DC: drbd02 (519baeac-3787-4f4a-8361-fb3e1f84cb21) - partition with quorum
Version: 1.0.13-a83fae5
2 Nodes configured, unknown expected votes
2 Resources configured.
============

Online: [ drbd01 drbd02 ]

 Master/Slave Set: ms_drbd0
     Masters: [ drbd01 drbd02 ]
 Resource Group: rg_drbd
     res_ip     (ocf::heartbeat:IPaddr):        Started FAILED [ drbd01 drbd02 ]
     res_filesystem     (ocf::heartbeat:Filesystem):    Started drbd01 (unmanaged) FAILED

Node Attributes:
* Node drbd01:
    + master-res_drbd0:0                : 10000
    + drbd02-eth0                  : up
* Node drbd02:
    + master-res_drbd0:1                : 10000
    + drbd01-eth0                  : up

Failed actions:
    res_filesystem_stop_0 (node=drbd01, call=21, rc=1, status=complete): unknown error
    res_ip_monitor_10000 (node=drbd02, call=12, rc=7, status=complete): not running
```

* Current DC: drbd02
    * drbd02がクラスタの指揮をとっている
    * クラスタに参加しているサーバのどれか(ここではdrbd01,drbd02)であればOK
* Masters: [ drbd01 drbd02 ]
    * drbd01, drbd02両方ともMasterになっている状態